## Import Modules

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
from os import listdir
from os.path import isfile, join
import time
import math
from IPython.display import clear_output
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import torchvision.transforms as T
#from torchsampler import ImbalancedDatasetSampler
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print('cuda available: '+ str(torch.cuda.is_available()))
#from skimage.morphology import disk, binary_dilation

from models_avm import NestedUNet
from loss_fun_avm import compute_per_channel_dice, DiceLoss, FocalLoss
from tra_val_avm import train, validation
from data_loader_avm import Dataset

cuda available: True


## Data Path

In [2]:
path_tra1 = '/work/samhong833/Data_AVM/forUNetpp/1_tra/1'
list_tra1 = os.listdir(path_tra1)
for i in range(len(list_tra1)):
    list_tra1[i] = path_tra1+'/'+list_tra1[i]
    
list_tra = list_tra1
    
path_val1 = '/work/samhong833/Data_AVM/forUNetpp/2_val/1'
list_val1 = os.listdir(path_val1)
for i in range(len(list_val1)):
    list_val1[i] = path_val1+'/'+list_val1[i]
    
list_val = list_val1

path_ts1 = '/work/samhong833/Data_AVM/forUNetpp/3_ts/1'
list_ts1 = os.listdir(path_ts1)
for i in range(len(list_ts1)):
    list_ts1[i] = path_ts1+'/'+list_ts1[i]
    
list_ts = list_ts1

path_tra_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/1_tra'
path_val_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/2_val'
path_ts_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/3_ts'

## Data Loader

In [3]:
train_data = Dataset(list_tra,path_tra_lab_txt,rand_dilate=True,max_dilate_factor=20)
train_loader = torch.utils.data.DataLoader(
    train_data, 
    batch_size = 12,
    shuffle = True,   
)

val_data = Dataset(list_val,path_val_lab_txt,rand_dilate=True,max_dilate_factor=20)
val_loader = torch.utils.data.DataLoader(
    dataset = val_data,           
    batch_size = 4,                 
    shuffle = False,              
)

## Initialize Model and Optimization Parameters

In [4]:
# call model cuda for gpu
model = NestedUNet().cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
kwargs = {"alpha": 0.5, "gamma": 3, "reduction": 'mean'}
criterion_FL = FocalLoss(**kwargs)
criterion_DICE = DiceLoss()
loss = [criterion_FL,criterion_DICE]

## Start to Train

In [ ]:
# Create Directory
path = '/work/samhong833/Models/Seg_dia20'

if os.path.isdir(path)==False:
    os.mkdir(path)
path = os.path.join(path,"train")
if os.path.isdir(path)==False:
    os.mkdir(path)       
filenum = glob.glob(path + "/exp*")
path = path + "/exp" + str(len(filenum)+1)
os.mkdir(path)

# Train the Model
epochs = 500 # The number of epochs

valloss = 0
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(path=path,
          model=model,
          loss=loss,
          optimizer=optimizer,
          dataloader=train_loader,
          epoch=epoch,
          scheduler=scheduler)
    print('-' * 89)
    vallossnew = validation(path=path,
          model=model,
          loss=loss,
          dataloader=val_loader,
          epoch=epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
    print('-' * 89)
    
    scheduler.step()
    
    f2 = open(path + '/model_info.txt', 'a')
    if vallossnew<valloss or epoch ==1: 
        fname = path + '/best_val'  + '.tar'
        torch.save(model.state_dict(), fname)
        valloss = vallossnew
        f2.write('| best_val | epoch {:3d}| '.format(epoch)+'\r\n')        
    f2.close()
    

| epoch   1 |   100/  290 batches | lr 0.0001000 | ms/batch 122663.14 | loss_fl 0.001439 | loss_dice  0.25 | 
| epoch   1 |   200/  290 batches | lr 0.0001000 | ms/batch 122496.77 | loss_fl 0.000955 | loss_dice  0.20 | 
| epoch   1 |   290/  290 batches | lr 0.0001000 | ms/batch 105758.46 | loss_fl 0.000790 | loss_dice  0.18 | 
-----------------------------------------------------------------------------------------
| epoch   1 |   100/  118 batches | | ms/batch 25209.25 | val_loss_fl 0.000508 | val_loss_dice  0.14 | 
| epoch   1 |   118/  118 batches | | ms/batch 4456.26 | val_loss_fl 0.000507 | val_loss_dice  0.13 | 
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 380.59s
-----------------------------------------------------------------------------------------
| epoch   2 |   100/  290 batches | lr 0.0001000 | ms/batch 108631.49 | loss_fl 0.000395 | loss_dice  0.13 | 
| epoch   2 |   200/  290 batches | lr 0.0001000

| epoch  11 |   100/  290 batches | lr 0.0000950 | ms/batch 105596.40 | loss_fl 0.000258 | loss_dice  0.09 | 
| epoch  11 |   200/  290 batches | lr 0.0000950 | ms/batch 105418.79 | loss_fl 0.000257 | loss_dice  0.09 | 
| epoch  11 |   290/  290 batches | lr 0.0000950 | ms/batch 95205.84 | loss_fl 0.000256 | loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| epoch  11 |   100/  118 batches | | ms/batch 17417.69 | val_loss_fl 0.000470 | val_loss_dice  0.11 | 
| epoch  11 |   118/  118 batches | | ms/batch 3038.62 | val_loss_fl 0.000461 | val_loss_dice  0.10 | 
-----------------------------------------------------------------------------------------
| end of epoch  11 | time: 326.68s
-----------------------------------------------------------------------------------------
| epoch  12 |   100/  290 batches | lr 0.0000950 | ms/batch 105491.79 | loss_fl 0.000262 | loss_dice  0.10 | 
| epoch  12 |   200/  290 batches | lr 0.0000950 

| epoch  21 |   100/  290 batches | lr 0.0000902 | ms/batch 105050.70 | loss_fl 0.000194 | loss_dice  0.08 | 
| epoch  21 |   200/  290 batches | lr 0.0000902 | ms/batch 105895.93 | loss_fl 0.000192 | loss_dice  0.08 | 
| epoch  21 |   290/  290 batches | lr 0.0000902 | ms/batch 97128.70 | loss_fl 0.000192 | loss_dice  0.07 | 
-----------------------------------------------------------------------------------------
| epoch  21 |   100/  118 batches | | ms/batch 16908.17 | val_loss_fl 0.000425 | val_loss_dice  0.10 | 
| epoch  21 |   118/  118 batches | | ms/batch 3023.40 | val_loss_fl 0.000420 | val_loss_dice  0.10 | 
-----------------------------------------------------------------------------------------
| end of epoch  21 | time: 328.01s
-----------------------------------------------------------------------------------------
| epoch  22 |   100/  290 batches | lr 0.0000902 | ms/batch 105000.59 | loss_fl 0.000181 | loss_dice  0.07 | 
| epoch  22 |   200/  290 batches | lr 0.0000902 

| epoch  31 |   100/  290 batches | lr 0.0000857 | ms/batch 105452.93 | loss_fl 0.000147 | loss_dice  0.06 | 
| epoch  31 |   200/  290 batches | lr 0.0000857 | ms/batch 111026.31 | loss_fl 0.000148 | loss_dice  0.06 | 
| epoch  31 |   290/  290 batches | lr 0.0000857 | ms/batch 102433.65 | loss_fl 0.000151 | loss_dice  0.06 | 
-----------------------------------------------------------------------------------------
| epoch  31 |   100/  118 batches | | ms/batch 19834.69 | val_loss_fl 0.000746 | val_loss_dice  0.10 | 
| epoch  31 |   118/  118 batches | | ms/batch 3564.49 | val_loss_fl 0.000731 | val_loss_dice  0.10 | 
-----------------------------------------------------------------------------------------
| end of epoch  31 | time: 342.32s
-----------------------------------------------------------------------------------------
| epoch  32 |   100/  290 batches | lr 0.0000857 | ms/batch 106293.77 | loss_fl 0.000143 | loss_dice  0.06 | 
| epoch  32 |   200/  290 batches | lr 0.0000857

| epoch  41 |   100/  290 batches | lr 0.0000815 | ms/batch 105025.76 | loss_fl 0.000116 | loss_dice  0.05 | 
| epoch  41 |   200/  290 batches | lr 0.0000815 | ms/batch 105278.92 | loss_fl 0.000113 | loss_dice  0.05 | 
| epoch  41 |   290/  290 batches | lr 0.0000815 | ms/batch 95001.20 | loss_fl 0.000111 | loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| epoch  41 |   100/  118 batches | | ms/batch 16783.20 | val_loss_fl 0.000937 | val_loss_dice  0.10 | 
| epoch  41 |   118/  118 batches | | ms/batch 3012.01 | val_loss_fl 0.000913 | val_loss_dice  0.10 | 
-----------------------------------------------------------------------------------------
| end of epoch  41 | time: 325.10s
-----------------------------------------------------------------------------------------
| epoch  42 |   100/  290 batches | lr 0.0000815 | ms/batch 105207.19 | loss_fl 0.000105 | loss_dice  0.05 | 
| epoch  42 |   200/  290 batches | lr 0.0000815 

...

| epoch 171 |   100/  290 batches | lr 0.0000418 | ms/batch 104962.77 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 171 |   200/  290 batches | lr 0.0000418 | ms/batch 105564.22 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 171 |   290/  290 batches | lr 0.0000418 | ms/batch 94770.29 | loss_fl 0.000020 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 171 |   100/  118 batches | | ms/batch 16848.61 | val_loss_fl 0.002895 | val_loss_dice  0.11 | 
| epoch 171 |   118/  118 batches | | ms/batch 3022.93 | val_loss_fl 0.002889 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 171 | time: 325.17s
-----------------------------------------------------------------------------------------
| epoch 172 |   100/  290 batches | lr 0.0000418 | ms/batch 105100.56 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 172 |   200/  290 batches | lr 0.0000418 

| epoch 181 |   100/  290 batches | lr 0.0000397 | ms/batch 104904.26 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 181 |   200/  290 batches | lr 0.0000397 | ms/batch 105080.22 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 181 |   290/  290 batches | lr 0.0000397 | ms/batch 94399.82 | loss_fl 0.000018 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 181 |   100/  118 batches | | ms/batch 17070.78 | val_loss_fl 0.003086 | val_loss_dice  0.11 | 
| epoch 181 |   118/  118 batches | | ms/batch 2990.39 | val_loss_fl 0.003040 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 181 | time: 324.45s
-----------------------------------------------------------------------------------------
| epoch 182 |   100/  290 batches | lr 0.0000397 | ms/batch 104858.62 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 182 |   200/  290 batches | lr 0.0000397 

| epoch 191 |   100/  290 batches | lr 0.0000377 | ms/batch 104880.05 | loss_fl 0.000017 | loss_dice  0.01 | 
| epoch 191 |   200/  290 batches | lr 0.0000377 | ms/batch 105487.65 | loss_fl 0.000017 | loss_dice  0.01 | 
| epoch 191 |   290/  290 batches | lr 0.0000377 | ms/batch 94133.29 | loss_fl 0.000017 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 191 |   100/  118 batches | | ms/batch 16619.92 | val_loss_fl 0.002990 | val_loss_dice  0.11 | 
| epoch 191 |   118/  118 batches | | ms/batch 2981.21 | val_loss_fl 0.002978 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 191 | time: 324.11s
-----------------------------------------------------------------------------------------
| epoch 192 |   100/  290 batches | lr 0.0000377 | ms/batch 105095.53 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 192 |   200/  290 batches | lr 0.0000377 

| epoch 201 |   100/  290 batches | lr 0.0000358 | ms/batch 110822.98 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 201 |   200/  290 batches | lr 0.0000358 | ms/batch 112200.03 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 201 |   290/  290 batches | lr 0.0000358 | ms/batch 100966.00 | loss_fl 0.000015 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 201 |   100/  118 batches | | ms/batch 22058.28 | val_loss_fl 0.003211 | val_loss_dice  0.11 | 
| epoch 201 |   118/  118 batches | | ms/batch 3784.69 | val_loss_fl 0.003170 | val_loss_dice  0.10 | 
-----------------------------------------------------------------------------------------
| end of epoch 201 | time: 349.84s
-----------------------------------------------------------------------------------------
| epoch 202 |   100/  290 batches | lr 0.0000358 | ms/batch 108169.82 | loss_fl 0.000015 | loss_dice  0.01 | 
| epoch 202 |   200/  290 batches | lr 0.0000358

| epoch 211 |   100/  290 batches | lr 0.0000341 | ms/batch 106554.64 | loss_fl 0.000014 | loss_dice  0.01 | 
| epoch 211 |   200/  290 batches | lr 0.0000341 | ms/batch 107931.65 | loss_fl 0.000014 | loss_dice  0.01 | 
| epoch 211 |   290/  290 batches | lr 0.0000341 | ms/batch 99345.08 | loss_fl 0.000014 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 211 |   100/  118 batches | | ms/batch 18294.69 | val_loss_fl 0.003249 | val_loss_dice  0.11 | 
| epoch 211 |   118/  118 batches | | ms/batch 3329.64 | val_loss_fl 0.003218 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 211 | time: 335.46s
-----------------------------------------------------------------------------------------
| epoch 212 |   100/  290 batches | lr 0.0000341 | ms/batch 105815.29 | loss_fl 0.000014 | loss_dice  0.01 | 
| epoch 212 |   200/  290 batches | lr 0.0000341 

# Log ^